In [67]:
import pandas as pd
import numpy as np
import math

import seaborn as sns
import matplotlib.pyplot as plt

# County Mapping

The counties_df will serve to assign human readable names to regions identified in the covid-19 timeseries dataset.

In [68]:
counties_df = pd.read_csv("../COVID-19/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv")


In [69]:
counties_df[covid_df['Admin2'] == "Charleston"]

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
3204,84045019,US,USA,840.0,45019.0,Charleston,South Carolina,US,32.824879,-79.965123,"Charleston, South Carolina, US",411406.0


# Confiremd Infections Timeseries

The confirmed infections timeseries provides daily updates on the total confirmed case counts in a region.

In [70]:
infections_df = pd.read_csv("../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
infections_df.head(1)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271,-170.132,...,0,0,0,0,0,0,0,0,0,0


In [71]:
#Dropping rows that do not have valid FIPS, as they are larger regions formed of several FIPS areas.
infections_df = infections_df.dropna(0, subset=['FIPS'])

In [72]:
# Sorting columns is important before diff'ing them. So that the 3rd of the month isn't afer the 29th.
# Padding the dates with 0's up to a length of 2 resolves the sort order mishap.
infections_df.columns = ["/".join([x.zfill(2) for x in n.split('/')]) for n  in infections_df.columns]
infections_df = infections_df.reindex(sorted(infections_df.columns), axis=1)


In [73]:
# Removing the columns that don't represent the timeseries
daily_new_confirmed_df = infections_df[infections_df.columns.difference(['UID', 'iso3', 'code3','Admin2',
                                                                         'Province_State', 'Country_Region', 
                                                                        'Lat','Long_', 'Combined_Key'])
                                      ].set_index('FIPS').diff(axis=1)


In [74]:
#These columns are then re-attached to the delta's
infection_locations = infections_df[['UID', 'iso3', 'code3','Admin2', 'Province_State', 'Country_Region',
                'Lat','Long_', 'Combined_Key', 'FIPS']].set_index('FIPS')
counties_daily_infections_df = infection_locations.join(daily_new_confirmed_df, how="left")
counties_daily_infections_df.index = counties_daily_infections_df.index.map(int)
counties_daily_infections_df

,UID,iso3,code3,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,01/22/20,...,06/21/20,06/22/20,06/23/20,06/24/20,06/25/20,06/26/20,06/27/20,06/28/20,06/29/20,iso2
FIPS,,,,,,,,,,,,,,,,,,,,,
60,16,ASM,16,NaN,American Samoa,US,-14.271000,-170.132000,"American Samoa, US",NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
66,316,GUM,316,NaN,Guam,US,13.444300,144.793700,"Guam, US",NaN,...,0.0,0.0,3.0,1.0,5.0,16.0,0.0,0.0,6.0,NaN
69,580,MNP,580,NaN,Northern Mariana Islands,US,15.097900,145.673900,"Northern Mariana Islands, US",NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
72,630,PRI,630,NaN,Puerto Rico,US,18.220800,-66.590100,"Puerto Rico, US",NaN,...,62.0,39.0,121.0,135.0,57.0,45.0,144.0,123.0,61.0,NaN
78,850,VIR,850,NaN,Virgin Islands,US,18.335800,-64.896300,"Virgin Islands, US",NaN,...,3.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,NaN
1001,84001001,USA,840,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",NaN,...,3.0,8.0,11.0,16.0,10.0,9.0,10.0,5.0,24.0,NaN
1003,84001003,USA,840,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",NaN,...,10.0,7.0,13.0,14.0,13.0,38.0,39.0,20.0,67.0,NaN
1005,84001005,USA,840,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",NaN,...,0.0,5.0,3.0,8.0,16.0,6.0,5.0,0.0,5.0,NaN
1007,84001007,USA,840,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",NaN,...,1.0,2.0,6.0,6.0,8.0,4.0,8.0,1.0,3.0,NaN


In [75]:
infection_delta_sc = counties_daily_infections_df[counties_daily_infections_df['Province_State'] == 'South Carolina']

In [76]:
infection_delta_sc

,UID,iso3,code3,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,01/22/20,...,06/21/20,06/22/20,06/23/20,06/24/20,06/25/20,06/26/20,06/27/20,06/28/20,06/29/20,iso2
FIPS,,,,,,,,,,,,,,,,,,,,,
45001,84045001,USA,840,Abbeville,South Carolina,US,34.223334,-82.461707,"Abbeville, South Carolina, US",NaN,...,2.0,-1.0,-1.0,5.0,8.0,1.0,3.0,0.0,3.0,NaN
45003,84045003,USA,840,Aiken,South Carolina,US,33.543380,-81.636454,"Aiken, South Carolina, US",NaN,...,7.0,5.0,2.0,21.0,12.0,11.0,10.0,10.0,13.0,NaN
45005,84045005,USA,840,Allendale,South Carolina,US,32.988374,-81.353211,"Allendale, South Carolina, US",NaN,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN
45007,84045007,USA,840,Anderson,South Carolina,US,34.518281,-82.639595,"Anderson, South Carolina, US",NaN,...,18.0,20.0,22.0,22.0,3.0,17.0,27.0,11.0,15.0,NaN
45009,84045009,USA,840,Bamberg,South Carolina,US,33.219276,-81.056600,"Bamberg, South Carolina, US",NaN,...,6.0,0.0,1.0,9.0,3.0,4.0,1.0,7.0,12.0,NaN
45011,84045011,USA,840,Barnwell,South Carolina,US,33.266793,-81.435844,"Barnwell, South Carolina, US",NaN,...,0.0,1.0,2.0,4.0,1.0,2.0,4.0,2.0,2.0,NaN
45013,84045013,USA,840,Beaufort,South Carolina,US,32.392263,-80.726198,"Beaufort, South Carolina, US",NaN,...,41.0,27.0,27.0,40.0,32.0,29.0,54.0,59.0,27.0,NaN
45015,84045015,USA,840,Berkeley,South Carolina,US,33.202226,-79.946545,"Berkeley, South Carolina, US",NaN,...,36.0,45.0,51.0,45.0,65.0,49.0,38.0,57.0,58.0,NaN
45017,84045017,USA,840,Calhoun,South Carolina,US,33.669269,-80.777222,"Calhoun, South Carolina, US",NaN,...,7.0,4.0,1.0,0.0,8.0,-3.0,-1.0,4.0,1.0,NaN


In [ ]:
plt.figure(figsize=(5,5))
sns.set(style="whitegrid")

infectionDeltaLong = pd.melt(infection_delta_sc.drop(['UID', 'iso2', 'iso3', 'code3','Admin2', 'Province_State', 'Country_Region',
                'Lat','Long_', 'Combined_Key'], axis=1).reset_index(), id_vars=['FIPS'], var_name=['Date'], value_name='Infected')
infectionDeltaLong = infectionDeltaLong.dropna(0,subset=['Infected'])
g = sns.FacetGrid(data=infectionDeltaLong, col='Date', col_wrap=5)
g.map(plt.bar, 'FIPS', 'Infected');


<Figure size 360x360 with 0 Axes>

In [ ]:
pd.set_option("display.max_rows", None)

infectionDeltaLong = pd.melt(infection_delta_sc.drop(['UID', 'iso2', 'iso3', 'code3','Admin2', 'Province_State', 'Country_Region',
                'Lat','Long_', 'Combined_Key'], axis=1).reset_index(), id_vars=['FIPS'], var_name=['Date'], value_name='Infected')
infectionDeltaLong.tail(100)